In [1]:
# Install necessary libraries
!pip install torch torchvision

# Download ModelNet40 dataset (optional: You can directly upload the dataset to Google Drive or Colab)
!wget https://modelnet.cs.princeton.edu/ModelNet40.zip
!unzip ModelNet40.zip -d data/


Streaming output truncated to the last 5000 lines.
  inflating: data/ModelNet40/monitor/train/monitor_0120.off  
  inflating: data/ModelNet40/monitor/train/monitor_0199.off  
  inflating: data/ModelNet40/monitor/train/monitor_0285.off  
  inflating: data/ModelNet40/monitor/train/monitor_0227.off  
  inflating: data/ModelNet40/monitor/train/monitor_0281.off  
  inflating: data/ModelNet40/monitor/train/monitor_0067.off  
  inflating: data/ModelNet40/monitor/train/monitor_0368.off  
  inflating: data/ModelNet40/monitor/train/monitor_0090.off  
  inflating: data/ModelNet40/monitor/train/monitor_0436.off  
  inflating: data/ModelNet40/monitor/train/monitor_0137.off  
  inflating: data/ModelNet40/monitor/train/monitor_0011.off  
  inflating: data/ModelNet40/monitor/train/monitor_0288.off  
  inflating: data/ModelNet40/monitor/train/monitor_0284.off  
  inflating: data/ModelNet40/monitor/train/monitor_0325.off  
  inflating: data/ModelNet40/monitor/train/monitor_0378.off  
  inflating: data/M

In [1]:
pip install trimesh

In [2]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
import trimesh
from torchvision import transforms

# Set device to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [28]:
class PointCloudDataset(Dataset):
    def __init__(self, root_dir, split='train', transform=None):
        self.root_dir = root_dir
        self.split = split
        self.transform = transform
        self.file_paths = []
        self.labels = []
        self.categories = sorted(os.listdir(root_dir))
        self.category_to_idx = {category: idx for idx, category in enumerate(self.categories)}

        self.load_data()

    def load_data(self):
        for category in self.categories:
            category_dir = os.path.join(self.root_dir, category, self.split)
            if os.path.isdir(category_dir):
                for file in os.listdir(category_dir):
                    if file.endswith('.off'):
                        self.file_paths.append(os.path.join(category_dir, file))
                        self.labels.append(self.category_to_idx[category])

    def __len__(self):
        return len(self.file_paths)

    def __getitem__(self, idx):
        file_path = self.file_paths[idx]
        label = self.labels[idx]

        mesh = trimesh.load(file_path)
        point_cloud = mesh.sample(768)  # Increased to 768 points

        if self.transform:
            point_cloud = self.transform(point_cloud)

        point_cloud = point_cloud - point_cloud.mean(axis=0)
        point_cloud = point_cloud / np.max(np.linalg.norm(point_cloud, axis=1))

        return torch.from_numpy(point_cloud).float().transpose(0, 1), label



In [29]:
class RandomScale:
    def __init__(self, scale_range=(0.8, 1.2)):
        self.scale_range = scale_range

    def __call__(self, point_cloud):
        scale = np.random.uniform(*self.scale_range)
        return point_cloud * scale

class JitterPoints:
    def __init__(self, sigma=0.01, clip=0.05):
        self.sigma = sigma
        self.clip = clip

    def __call__(self, point_cloud):
        jittered_data = np.clip(self.sigma * np.random.randn(*point_cloud.shape), -self.clip, self.clip)
        return point_cloud + jittered_data

transform = transforms.Compose([RandomRotation(), RandomScale(), JitterPoints()])


In [30]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
import numpy as np
import os
import trimesh
from torchvision import transforms
from tqdm import tqdm  # For progress bars


In [31]:
batch_size = 16

train_dataset = PointCloudDataset(root_dir='data/ModelNet40', split='train', transform=transform)
test_dataset = PointCloudDataset(root_dir='data/ModelNet40', split='test', transform=None)

# Using only 1000 samples for training and 200 for testing (for debugging)
train_dataset, _ = random_split(train_dataset, [1000, len(train_dataset) - 1000])
test_dataset, _ = random_split(test_dataset, [200, len(test_dataset) - 200])

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


In [32]:
class PointNet(nn.Module):
    def __init__(self, num_classes):
        super(PointNet, self).__init__()
        self.conv1 = nn.Conv1d(3, 64, kernel_size=1)
        self.bn1 = nn.BatchNorm1d(64)  # Batch Normalization
        self.conv2 = nn.Conv1d(64, 128, kernel_size=1)
        self.bn2 = nn.BatchNorm1d(128)
        self.conv3 = nn.Conv1d(128, 512, kernel_size=1)  # Increased to 512 channels
        self.bn3 = nn.BatchNorm1d(512)

        self.fc1 = nn.Linear(512, 256)
        self.bn4 = nn.BatchNorm1d(256)
        self.fc2 = nn.Linear(256, 128)
        self.bn5 = nn.BatchNorm1d(128)
        self.fc3 = nn.Linear(128, num_classes)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.3)
        self.log_softmax = nn.LogSoftmax(dim=1)

    def forward(self, x):
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.relu(self.bn2(self.conv2(x)))
        x = self.relu(self.bn3(self.conv3(x)))
        x = torch.max(x, 2)[0]  # Max pooling
        x = self.relu(self.bn4(self.fc1(x)))
        x = self.dropout(self.relu(self.bn5(self.fc2(x))))
        x = self.fc3(x)
        return self.log_softmax(x)


In [33]:
def train(model, loader, optimizer, criterion, device):
    model.train()
    total_loss = 0
    correct = 0
    total = 0

    for data, target in tqdm(loader, desc="Training"):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        pred = output.argmax(dim=1)
        correct += pred.eq(target).sum().item()
        total += target.size(0)

    return total_loss / len(loader), correct / total

def test(model, loader, criterion, device):
    model.eval()
    total_loss = 0
    correct = 0
    total = 0

    with torch.no_grad():
        for data, target in tqdm(loader, desc="Testing"):
            data, target = data.to(device), target.to(device)
            output = model(data)
            loss = criterion(output, target)
            total_loss += loss.item()
            pred = output.argmax(dim=1)
            correct += pred.eq(target).sum().item()
            total += target.size(0)

    return total_loss / len(loader), correct / total


In [34]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = PointNet(num_classes=len(train_dataset.dataset.categories)).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.0005, weight_decay=1e-4)  # Reduced learning rate, added weight decay
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)
criterion = nn.CrossEntropyLoss()

epochs = 20
for epoch in range(epochs):
    print(f'Epoch {epoch+1}/{epochs}')

    train_loss, train_acc = train(model, train_loader, optimizer, criterion, device)
    test_loss, test_acc = test(model, test_loader, criterion, device)
    scheduler.step()

    print(f'Epoch {epoch+1}/{epochs} | Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f} | Test Loss: {test_loss:.4f} | Test Acc: {test_acc:.4f}')


Epoch 1/20


Testing: 100%|██████████| 13/13 [00:28<00:00,  2.22s/it]


Epoch 1/20 | Train Loss: 3.1588 | Train Acc: 0.2160 | Test Loss: 2.6465 | Test Acc: 0.2550
Epoch 2/20


Testing: 100%|██████████| 13/13 [00:28<00:00,  2.19s/it]


Epoch 2/20 | Train Loss: 2.4686 | Train Acc: 0.3840 | Test Loss: 2.4841 | Test Acc: 0.3650
Epoch 3/20


...

Epoch 18/20 | Train Loss: 0.8417 | Train Acc: 0.7740 | Test Loss: 1.0544 | Test Acc: 0.7150
Epoch 19/20


Testing: 100%|██████████| 13/13 [00:27<00:00,  2.11s/it]


Epoch 19/20 | Train Loss: 0.8383 | Train Acc: 0.7720 | Test Loss: 1.0835 | Test Acc: 0.7100
Epoch 20/20


Testing: 100%|██████████| 13/13 [00:27<00:00,  2.10s/it]

Epoch 20/20 | Train Loss: 0.8126 | Train Acc: 0.7740 | Test Loss: 1.1122 | Test Acc: 0.7050
